In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy import ndimage
from func import *
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from matplotlib.colors import LogNorm

In [2]:
def err_ratio(data,err):   
    (y_blr,x_blr) = ndimage.measurements.maximum_position(data)
    (act_dat,act_err) = (data,err)
    data_mask = data[1:-1,1:-1]
    err_mask = err[1:-1,1:-1]
    data[1:-1,1:-1]=0
    err[1:-1,1:-1]=0
    data_edge = data[data!=0]
    err_edge = err[err!=0]
    err_2D_dat = np.std(data_edge)
    err_2D_err = np.mean(err_edge)
    correct_fact = np.abs(err_2D_dat/err_2D_err)
    return correct_fact

def flux_err_fact(obj,destination_path_cube="/home/rickeythecat/Downloads/Outflow_paper1/MUSE"):
    hdu = fits.open('%s/%s/%s_9_arcsec_empirical_factor_subcube.fits'%(destination_path_cube,obj,obj))
    central_tab = hdu[1].data
    central_columns = hdu[1].header
    (emp_Hb1,emp_Hb2,emp_wing) = (central_tab.field('emp_fact_Hb1')[0],central_tab.field('emp_fact_Hb2')[0],central_tab.field('emp_fact_wing')[0])
    hdu.close()
    return emp_Hb1,emp_Hb2,emp_wing
    
def flux_error_single_comp(data,err,vel_sigma,vel_sigma_err,ratio,flux):
    identity_array = (data/data)
    amp_contribution = (err/data)
    true_amp_contribution = amp_contribution*ratio
    vel_sigma_contribution = 0.5*(vel_sigma_err/vel_sigma)*identity_array
    #print vel_sigma_contribution,true_amp_contribution 
    flux_err= flux*(true_amp_contribution + vel_sigma_contribution)
    return flux_err

def blr_flux(flux_Hb1,flux_Hb2,broad2=False):
    if broad2:
        flux_Hb_blr = flux_Hb1+flux_Hb2
    else:
        flux_Hb_blr = flux_Hb1
    return flux_Hb_blr 

def blr_flux_err(flux_Hb1_err,flux_Hb2_err,broad2=False):
    if broad2:
        flux_Hb_blr_err = np.sqrt(flux_Hb1_err**2+flux_Hb2_err**2) 
    else:
        flux_Hb_blr_err = flux_Hb1_err
    return flux_Hb_blr_err 


In [3]:
def blr_flux_map(output_par,obj,destination_path_cube="/home/rickeythecat/Downloads/Outflow_paper1/MUSE"):
    [flux_Hb1,flux_Hb2,flux_Hb_blr] = output_par
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(flux_Hb1,name='flux_Hb1_blr'))
    hdus.append(fits.ImageHDU(flux_Hb2,name='flux_Hb2_blr'))
    hdus.append(fits.ImageHDU(flux_Hb_blr,name='flux_Hb_blr'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/fluxcube_blr_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')

def blr_flux_err_map(output_par_err,obj,destination_path_cube="/home/rickeythecat/Downloads/Outflow_paper1/MUSE"):
    [flux_Hb1_error,flux_Hb2_error,flux_Hb_blr_error] = output_par_err
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(flux_Hb1_error,name='flux_Hb1_blr_err'))
    hdus.append(fits.ImageHDU(flux_Hb2_error,name='flux_Hb2_blr_err'))
    hdus.append(fits.ImageHDU(flux_Hb_blr_error,name='flux_Hb_blr_err'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/fluxcube_blr_err_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')

def new_flux_err_map(obj,correction_fact,destination_path_cube="/home/rickeythecat/Downloads/Outflow_paper1/MUSE"):
    [Hb1_fact,Hb2_fact,Hb_fact] = correction_fact
    hdu = fits.open('%s/Flux Maps/%s/fluxcube_blr_err_%s.fits'%(destination_path_cube,obj,obj))
    Hb1_err = hdu[1].data
    Hb2_err = hdu[2].data
    Hb_err = hdu[3].data
    Hb1_err_new = Hb1_fact*Hb1_err
    Hb2_err_new = Hb2_fact*Hb2_err
    Hb_err_new = Hb_fact*Hb_err
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(Hb1_err,name='flux_Hb1_blr_err'))
    hdus.append(fits.ImageHDU(Hb2_err,name='flux_Hb2_blr_err'))
    hdus.append(fits.ImageHDU(Hb_err,name='flux_Hb_blr_err'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/fluxcube_blr_err_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')


In [4]:
def algorithm_script(obj,z,broad2,destination_path_cube="/home/rickeythecat/Downloads/Outflow_paper1/MUSE"):
    print ('%s'%(obj))

    (amp_Hb1,amp_Hb2,vel_Hb1,vel_Hb2,vel_sigma_Hb1,vel_sigma_Hb2,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,vel_offset,m,c) = central_par(obj)
    (vel_sigma_Hb1_err,vel_sigma_Hb2_err,vel_sigma_OIII_err,vel_sigma_OIII_br_err) = central_vel_sigma_err(obj)
    print ("Value and Error with BLR1 Velocity dispersion is",vel_sigma_Hb1,vel_sigma_Hb1_err)
    print ("Value and Error with BLR2 Velocity dispersion is",vel_sigma_Hb2,vel_sigma_Hb2_err)
    
    (Hb1_blr_br_data,Hb2_blr_br_data) = loadblr('%s/Flux Maps/%s/9_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj))
    (Hb1_blr_br_err,Hb2_blr_br_err) = loadblr('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    OIII_br_err = loadwing('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    OIII_br_data = loadwing('%s/Flux Maps/%s/9_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj)) # It opens the hdu[3] where the OIII_br data 
                                                        # whether that is error or data because it sepends upon filename
    sigma_eff_Hb1 = line_width_recons(vel_sigma_Hb1,rest_line=4861.33,inst_res_fwhm=0)
    sigma_eff_Hb2 = line_width_recons(vel_sigma_Hb2,rest_line=4861.33,inst_res_fwhm=0) 
    sigma_eff_OIII_br = line_width_recons(vel_sigma_OIII_br,rest_line=5006.8,inst_res_fwhm=0) 
    
    flux_Hb1 = Hb1_blr_br_data*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb1))
    flux_Hb2 = Hb2_blr_br_data*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb2))   
    flux_Hb_blr = blr_flux(flux_Hb1,flux_Hb2,broad2)
    
    output_par = [flux_Hb1,flux_Hb2,flux_Hb_blr]
    blr_flux_map(output_par,obj)

    (err_ratio_Hb1,err_ratio_Hb2,err_ratio_OIII_br) = flux_err_fact(obj)
    
    flux_Hb1_err = flux_error_single_comp(Hb1_blr_br_data,Hb1_blr_br_err,vel_sigma_Hb1,vel_sigma_Hb1_err,err_ratio_Hb1,flux_Hb1)
    flux_Hb2_err = flux_error_single_comp(Hb2_blr_br_data,Hb2_blr_br_err,vel_sigma_Hb2,vel_sigma_Hb2_err,err_ratio_Hb2,flux_Hb2) 
    flux_Hb_blr_err = blr_flux_err(flux_Hb1_err,flux_Hb2_err,broad2) 
    
    output_par_err = [flux_Hb1_err,flux_Hb2_err,flux_Hb_blr_err]
    blr_flux_err_map(output_par_err,obj)
   
    Hb1_err_correction_fact = err_ratio(flux_Hb1,flux_Hb1_err)
    Hb2_err_correction_fact = err_ratio(flux_Hb2,flux_Hb2_err)
    blr_err_correction_fact = err_ratio(flux_Hb_blr,flux_Hb_blr_err)
    #print blr_err_correction_fact
    new_err = [blr_err_correction_fact*output_par_err[0],blr_err_correction_fact*output_par_err[1],blr_err_correction_fact*output_par_err[2]]
    correction_fact = [Hb1_err_correction_fact,Hb2_err_correction_fact,blr_err_correction_fact]
    new_flux_err_map(obj,correction_fact)
 

In [5]:
z = {"HE0227-0913":0.016451}

broad2= {'HE0227-0913':True}

objs = z.keys()


for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj])

HE0227-0913
Value and Error with BLR1 Velocity dispersion is 397.35577 3.3429568
Value and Error with BLR2 Velocity dispersion is 1316.8972 8.997515


In [6]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

broad2= {'HE0021-1810':False,'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':False}

objs = z.keys()


for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj])

HE0021-1810
Value and Error with BLR1 Velocity dispersion is 2234.2498 89.995544
Value and Error with BLR2 Velocity dispersion is 1000.0 0.0


/home/rickeythecat/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/rickeythecat/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


HE0021-1819
Value and Error with BLR1 Velocity dispersion is 1599.3799 10.298232
Value and Error with BLR2 Velocity dispersion is 1000.0 0.0
HE0040-1105
Value and Error with BLR1 Velocity dispersion is 1194.5739 3.5906205
Value and Error with BLR2 Velocity dispersion is 1000.0 0.0
HE0108-4743
Value and Error with BLR1 Velocity dispersion is 1514.2646 107.41867
Value and Error with BLR2 Velocity dispersion is 464.72116 14.925348
HE0114-0015
Value and Error with BLR1 Velocity dispersion is 1198.4703 14.951251
Value and Error with BLR2 Velocity dispersion is 1000.0 0.0
HE0119-0118
Value and Error with BLR1 Velocity dispersion is 994.1203 53.3468
Value and Error with BLR2 Velocity dispersion is 2358.1272 75.138245
HE0212-0059
Value and Error with BLR1 Velocity dispersion is 3993.8176 1104.622
Value and Error with BLR2 Velocity dispersion is 1133.6825 1169.8607
HE0224-2834
Value and Error with BLR1 Velocity dispersion is 2227.0266 26.885946
Value and Error with BLR2 Velocity dispersion is 1

/home/rickeythecat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in float_scalars
/home/rickeythecat/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/rickeythecat/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/rickeythecat/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


HE1029-1831
Value and Error with BLR1 Velocity dispersion is 1678.7982 90.98273
Value and Error with BLR2 Velocity dispersion is 886.55975 117.03634
HE1107-0813
Value and Error with BLR1 Velocity dispersion is 1084.3862 166.31647
Value and Error with BLR2 Velocity dispersion is 2151.3716 576.54315
HE1108-2813
Value and Error with BLR1 Velocity dispersion is 1291.9475 5.819236
Value and Error with BLR2 Velocity dispersion is 1000.0 0.0
HE1126-0407
Value and Error with BLR1 Velocity dispersion is 704.07874 7.8642025
Value and Error with BLR2 Velocity dispersion is 1729.1505 11.786969
HE1237-0504
Value and Error with BLR1 Velocity dispersion is 1650.4827 4.0263705
Value and Error with BLR2 Velocity dispersion is 0.0 0.0
HE1248-1356
Value and Error with BLR1 Velocity dispersion is 1301.8562 28.114538
Value and Error with BLR2 Velocity dispersion is 0.0 0.0
HE1330-1013
Value and Error with BLR1 Velocity dispersion is 1661.5006 48.41305
Value and Error with BLR2 Velocity dispersion is 622.99

In [8]:
a = [1,2,3]
b = [4,5,6]
np.multiply(a,b)

array([ 4, 10, 18])